<a href="https://colab.research.google.com/github/Nutsudapenpong/DPDM/blob/master/Association_Rules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd

In [7]:
# connect with google drive
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/My Drive/DWDM_material'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import os

In [9]:
data = pd.read_csv(os.path.join(path, 'reduced_marketbasket.csv'), encoding='latin1') #มีอักษร latin ในไฟล์เลยต้องเพิ่ม encoding latin1
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/01/2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/01/2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/01/2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/01/2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/01/2010 08:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/09/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/09/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/09/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/09/2011 12:50,4.15,12680.0,France


In [10]:
data.groupby('InvoiceNo').count()

,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
InvoiceNo,,,,,,,
536365,7,7,7,7,7,7,7
536366,2,2,2,2,2,2,2
536367,12,12,12,12,12,12,12
536368,4,4,4,4,4,4,4
536369,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...
C581484,1,1,1,1,1,1,1
C581490,2,2,2,2,2,2,2
C581499,1,1,1,1,1,1,1


In [11]:
data.groupby('InvoiceNo').sum()

,Quantity,UnitPrice,CustomerID
InvoiceNo,,,
536365,40,27.37,124950.0
536366,12,3.70,35700.0
536367,83,58.24,156564.0
536368,15,19.10,52188.0
536369,3,5.95,13047.0
...,...,...,...
C581484,-80995,2.08,16446.0
C581490,-23,2.78,28794.0
C581499,-1,224.69,15498.0


In [12]:
transacs = data.groupby('InvoiceNo')['StockCode'].apply(list) #เอามาใส่ใน list โดยไม่ sum หรือ count
transacs

InvoiceNo
536365     [85123A, 71053, 84406B, 84029G, 84029E, 22752,...
536366                                        [22633, 22632]
536367     [84879, 22745, 22748, 22749, 22310, 84969, 226...
536368                          [22960, 22913, 22912, 22914]
536369                                               [21756]
                                 ...                        
C581484                                              [23843]
C581490                                       [22178, 23144]
C581499                                                  [M]
C581568                                              [21258]
C581569                                       [84978, 20979]
Name: StockCode, Length: 25900, dtype: object

Apriori

http://zaxrosenberg.com/unofficial-apyori-documentation/

In [13]:
!pip install apyori #ติดตั้งไลบราลีก่อน

  Created wheel for apyori: filename=apyori-1.1.2-cp36-none-any.whl size=5975 sha256=e0e70f5b50ea8216fb7f73c3bad17c6f2407d8b7367884baf592da00845edb2d
  Stored in directory: /root/.cache/pip/wheels/5d/92/bb/474bbadbc8c0062b9eb168f69982a0443263f8ab1711a8cad0
Successfully built apyori


In [14]:
from apyori import apriori

In [16]:
out = list(apriori(transacs, min_support = 0.03, min_confidence = 0.1)) #ครอบ list เพราะมันไม่ได้ออกมาแบบ list มันอ่านยาก

In [17]:
out #RelationRecord 
    #items ก็คือ คู่อันดับที่เราสนใจ
    #support ก็คือ ความน่าจะเป็นที่เกิดคู่อันดับที่เราสนใจ
    #ordered_statistics ซื้ออันแรกแล้วซื้ออันหลัง มี confident คือเปอร์เซ็นตามที่แสดง 
    #lift ดูความสัมพันธ์เมื่อกำหนดเงื่อนไข

[RelationRecord(items=frozenset({'22386', '85099B'}), support=0.03216216216216216, ordered_statistics=[OrderedStatistic(items_base=frozenset({'22386'}), items_add=frozenset({'85099B'}), confidence=0.6766856214459789, lift=8.20897311262335), OrderedStatistic(items_base=frozenset({'85099B'}), items_add=frozenset({'22386'}), confidence=0.39016393442622943, lift=8.20897311262335)]),
 RelationRecord(items=frozenset({'22699', '22697'}), support=0.03027027027027027, ordered_statistics=[OrderedStatistic(items_base=frozenset({'22697'}), items_add=frozenset({'22699'}), confidence=0.7417218543046358, lift=17.1523178807947), OrderedStatistic(items_base=frozenset({'22699'}), items_add=frozenset({'22697'}), confidence=0.7, lift=17.1523178807947)])]

Quiz หา k-itemset ที่มีความน่าสนใจ (โดยพิจารณาลูกค้าเป็นรายคน) พร้อมทั้งอธิบายว่าน่าสนใจยังไง

In [18]:
transacs_CustomerID = data.groupby('CustomerID')['StockCode'].apply(list) #เอามาใส่ใน list โดยไม่ sum หรือ count
transacs_CustomerID

CustomerID
12346.0                                       [23166, 23166]
12347.0    [85116, 22375, 71477, 22492, 22771, 22772, 227...
12348.0    [84992, 22951, 84991, 84991, 21213, 21213, 226...
12349.0    [23112, 23460, 21564, 21411, 21563, 22131, 221...
12350.0    [21908, 22412, 79066K, 79191C, 22348, 84086C, ...
                                 ...                        
18280.0    [82484, 22180, 22467, 22725, 22727, 22495, 223...
18281.0    [22037, 22716, 22028, 23007, 23008, 23209, 22467]
18282.0    [21270, 23187, 23295, 22089, 21108, 21109, 224...
18283.0    [22356, 20726, 22384, 22386, 20717, 20718, 850...
18287.0    [22755, 22754, 22753, 22756, 22758, 22757, 227...
Name: StockCode, Length: 4372, dtype: object

In [46]:
out_1 = list(apriori(transacs_CustomerID, min_support = 0.074, min_confidence = 0.5))

In [47]:
out_1

[RelationRecord(items=frozenset({'22910', '22086'}), support=0.07479414455626715, ordered_statistics=[OrderedStatistic(items_base=frozenset({'22086'}), items_add=frozenset({'22910'}), confidence=0.5317073170731708, lift=4.946009340944474), OrderedStatistic(items_base=frozenset({'22910'}), items_add=frozenset({'22086'}), confidence=0.6957446808510638, lift=4.946009340944474)]),
 RelationRecord(items=frozenset({'22699', '22423'}), support=0.07410795974382434, ordered_statistics=[OrderedStatistic(items_base=frozenset({'22699'}), items_add=frozenset({'22423'}), confidence=0.7587822014051523, lift=3.740017795426523)]),
 RelationRecord(items=frozenset({'22699', '22697'}), support=0.07410795974382434, ordered_statistics=[OrderedStatistic(items_base=frozenset({'22697'}), items_add=frozenset({'22699'}), confidence=0.8307692307692307, lift=8.506143037290578), OrderedStatistic(items_base=frozenset({'22699'}), items_add=frozenset({'22697'}), confidence=0.7587822014051523, lift=8.506143037290578)])

ความน่าจะเป็นที่คนจะซื้อ {'22910'} และ {'22086'} พร้อมกัน เท่ากับ 0.07479414455626715  และ ความน่าจะเป็นที่คนที่ซื้อ {'22910'} จะซื้อ {'22086'}  เท่ากับ 0.5317073170731708

ความน่าจะเป็นที่คนจะซื้อ {'22699'} และ {'22423'} พร้อมกัน เท่ากับ 0.07410795974382434 และ ความน่าจะเป็นที่คนที่ซื้อ {'22699'} จะซื้อ {'22423'}เท่ากับ 0.7587822014051523

ความน่าจะเป็นที่คนจะซื้อ {'22699'} และ {'22697'} พร้อมกัน เท่ากับ 0.07410795974382434 และ ความน่าจะเป็นที่คนที่ซื้อ {'22699'} จะซื้อ {'22697'}เท่ากับ 0.8307692307692307